In [24]:
import numpy as np
import pandas as pd

DUMMY_DATA_PATH = 'dataset/dummy/'
DUMMY_BANK_DATA = DUMMY_DATA_PATH+'BSA.csv'
DUMMY_MAIN_DATA = DUMMY_DATA_PATH+'data.csv'

FINAL_DATA = 'dataset/loan.csv'
TRAIN_DATA = 'dataset/loan_one_hot_encoded.csv'

In [25]:
PUBLIC_EMAIL_DOMAINS = ()
with open('public-email-domains.txt', 'r') as f:
    PUBLIC_EMAIL_DOMAINS = tuple(d.strip() for d in f.readlines())

In [26]:
main_df = pd.read_csv(DUMMY_MAIN_DATA)
bank_df = pd.read_csv(DUMMY_BANK_DATA)

In [27]:
to_remove_cols = [
     'address', # textual, and hence not helpful
     'ads_cmpid', # unique key
     'ads_creative', # unique key
#      'ads_matchtype',
#      'ads_network',
     'ads_targetid', # unique key
#      'amount',
#      'application_id',
     'birthdate', # use firm age (better fit to the model)
#      'browser',
#      'campaign_city',
#      'city',
#      'company_size',
     'country', # all are india as-is, so no point keeping it..
     'created_date', # has no correlation on the model.. 
#      'email',
     'firm_name', # each name is potentially unique, also string so can't do much
     'firm_pan', # unique for each company, NOTE: there is a pattern than can be used to extract features!! (TODO)
#      'firm_type',
#      'gender',
#      'industry',
     'ip', # can we do something about IPs? (TODO)
#      'last_fy_profit',
     'latitude', # IDK how helpful coordinates are.. We have the city and states; and so we can use that..
#      'loan_created', # THIS IS OUR TARGET! THIS IS SKEWED, SO A NAIVE IMPL WILL ALSO HAVE 96% ACCURACY :D 
     'longitude', # see comment for 'latitude'
     'name', # see comment for firm name
     'network', # don't need it.. too many random values.. 
     'pan', # unique for each individual, NOTE: there is a pattern than can be used to extract features!! (TODO)
     'pincode',
#      'platform',
#      'registered_office_city',
#      'registered_office_state',
#      'role_in_firm',
#      'role_on_application',
     'seo_city', # only 1 value.. 
#      'state',
#      'utm_medium',
#      'utm_source',
     'utm_term', # idk what to do with this data.. ¯\_(ツ)_/¯
#      'year_of_incorporation',
]

main_df = main_df.drop(columns=to_remove_cols)
main_df.head()

,ads_matchtype,ads_network,amount,application_id,browser,campaign_city,city,company_size,email,firm_type,...,loan_created,platform,registered_office_city,registered_office_state,role_in_firm,role_on_application,state,utm_medium,utm_source,year_of_incorporation
0,NaN,NaN,NaN,1024.0,Opera,NaN,NaN,NaN,Kaif1779@gmail.com,NaN,...,0,mobile:Pike v8.0 release 461,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1,NaN,NaN,300000.0,716.0,Chrome,NaN,NaN,NaN,vihanmarketing36@gmail.com,Proprietorship,...,1,Win32,Gondia,MAHARASHTRA,1.0,4,MAHARASHTRA,NaN,NaN,2014.0
2,NaN,NaN,200000.0,1031.0,Chrome,NaN,NaN,NaN,faijiyatoursandtravels@gmail.com,Proprietorship,...,0,mobile:Linux armv8l,PUNE,MAHARASHTRA,1.0,4,MAHARASHTRA,NaN,NaN,2016.0
3,e,{google_search},300000.0,2056.0,Chrome,Pune,Mumbai,5.0,sagarnk2008@gmail.com,Proprietorship,...,0,mobile:Linux aarch64,Pune,MAHARASHTRA,1.0,4,KARNATAKA,ppc,adwords,2014.0
4,NaN,NaN,500000.0,9047.0,Chrome,NaN,Mumbai,NaN,natrajmoily@gmail.com,Proprietorship,...,0,mobile:Linux armv8l,THANE,MAHARASHTRA,1.0,4,MAHARASHTRA,Banner,Facebook,2014.0


In [28]:
'''
Things to clean in main dataframe:
- [x] amount: make 0 amounts as NaNs
- [x] birthdate: calculate age of the person (do we need this if we have the age of firm?) --> removed col: using firm age
- [ ] browser: some really low counts
- [ ] campaign_city: some really low counts
- [ ] city: some really low counts
- [x] email: publicly hosted email domain or personal email domain
- [ ] firm_type: is skewed (need to figure things out..)
- [x] last_fy_profit: convert 0 to NaNs
- [x] platform: combine all the 'mobile:Linux'?
- [x] registered_office_city: combine same values (cases are different hence are treated as separate values)
- [ ] role_in_firm: categorical; so don't use the numbers as is..
- [ ] role_on_application: categorical; so don't use the numbers as is..
- [x] year_of_incorporation: -> compute age of firm
'''
# remove 0 amounts
main_df.loc[main_df['amount'] == 0, 'amount'] = np.NAN

# create a boolean column
main_df['private_email_domain'] = False
for index, row in main_df.iterrows():
    email = row['email']
    if email is not np.NaN and email.split('@')[1] not in PUBLIC_EMAIL_DOMAINS:
        main_df.loc[index, 'private_email_domain'] = True
main_df = main_df.drop(columns=['email']) # drop email column

# remove 0 last_fy_profit
main_df.loc[main_df['last_fy_profit'] == 0, 'last_fy_profit'] = np.NAN

# make all text uppercase in registered_office_city
main_df['registered_office_city'] = main_df['registered_office_city'].str.upper()

# compute age of firm
# main_df.loc[main_df['year_of_incorporation'] == 0, 'year_of_incorporation'] = np.NAN
main_df['age_of_firm'] = np.nan
def compute_age_of_firm(x):
    if x is np.nan:
        return np.nan
    elif type(x) == str:
        if '/' in x:
            x = x.split('/')[1].split('.')[0].strip('., ')
        x = x.split('.')[0].strip('., ')
        x = int(x)
        if 2018-x == 2018:
            return np.nan
        return 2018-x
    else:
        return x
main_df['age_of_firm'] = list(map(compute_age_of_firm, main_df['year_of_incorporation']))
main_df = main_df.drop(columns=['year_of_incorporation']) # drop email column

# strip the redundant brace brakets around the ads_network
main_df['ads_network'] = list(map(lambda x: x.strip('}{') if type(x) == str else x, main_df['ads_network']))

# platform: combine all the 'mobile:Linux'?
main_df['platform'] = list(map(lambda x: 'mobile:Linux' if type(x) == str and 'mobile:Linux' in x else x, main_df['platform']))

print(len(main_df), len(list(main_df)))

432 23


In [29]:
# # only keep applications that are in both data sets
# appln_id = pd.Series(list(set(main_df['application_id']) & set(bank_df['application_id'])))
# main_df = main_df.loc[main_df['application_id'].isin(appln_id)]
# bank_df = bank_df.loc[bank_df['application_id'].isin(appln_id)]

In [30]:
# list(bank_df)

In [31]:
def _aggregate_columns(df, application_id_col):
    # group by application id and merge all rows into lists
    new_df = pd.DataFrame()
    g = bank_df.groupby(application_id_col)
    for k in list(df):
        if k == application_id_col:
            continue
        new_df = pd.concat([new_df, g[k].apply(list)], axis=1)
    return new_df.reset_index()


def setup_aggregations(df, application_id_col):
    '''
    fix bank data (for applications with multiple rows)
    - average the averages
    - add high_credit_cp
    - add invard returns
    - max of all the maxs
    - min of all the mins
    - add outward_returns
    - drop totals (because average is better and normalized)
    '''
    df = df.drop(columns=['total_business_inflow', 'total_business_outflow', 'total_inflow', 'total_outflow'])
    df = _aggregate_columns(df, application_id_col)
    new_df = pd.DataFrame()
    for k in list(df):
        if k == application_id_col:
            new_df = pd.concat([new_df, df[k]], axis=1)
        elif 'average' in k:
            new_df = pd.concat([new_df, df[k].apply(np.average)], axis=1)
        elif 'max' in k:
            new_df = pd.concat([new_df, df[k].apply(np.max)], axis=1)
        elif 'min' in k:
            new_df = pd.concat([new_df, df[k].apply(np.min)], axis=1)
        else:
            new_df = pd.concat([new_df, df[k].apply(np.sum)], axis=1)
    return new_df

bank_df = setup_aggregations(bank_df, 'application_id')

In [32]:
print(len(bank_df), len(list(bank_df)))

230 16


In [33]:
df = pd.merge(main_df, bank_df, on='application_id')

In [34]:
print(len(df), len(list(df)))

230 38


In [35]:
df.head()

,ads_matchtype,ads_network,amount,application_id,browser,campaign_city,city,company_size,firm_type,gender,...,inward_returns,max_business_inflow,max_business_outflow,max_inflow,max_outflow,min_business_inflow,min_business_outflow,min_inflow,min_outflow,outward_returns
0,NaN,NaN,200000.0,1031,Chrome,NaN,NaN,NaN,Proprietorship,Male,...,2,502725,570348,502725,502725,35981,24331,35981,35981,0
1,e,google_search,300000.0,2056,Chrome,Pune,Mumbai,5.0,Proprietorship,Male,...,0,159971,159356,159971,159971,43826,47157,43826,43826,0
2,NaN,NaN,500000.0,9047,Chrome,NaN,Mumbai,NaN,Proprietorship,Male,...,0,134835,133462,134835,134835,0,0,0,0,0
3,NaN,google_display,500000.0,2068,Chrome,NaN,Mumbai,5.0,Proprietorship,Male,...,0,879035,780395,879035,879035,21,31236,21,21,0
4,b,google_search,500000.0,2737,Chrome,Jaipur,Ajmer,5.0,Proprietorship,Male,...,0,373105,285950,373105,373105,22000,30008,22000,22000,3


In [36]:
df.to_csv(FINAL_DATA, index=False)

In [37]:
categorical_cols = [
 'ads_matchtype',
 'ads_network',
#  'amount',
#  'application_id',
 'browser',
 'campaign_city',
 'city',
#  'company_size',
 'firm_type',
 'gender',
 'industry',
#  'last_fy_profit',
#  'loan_created',
 'platform',
 'registered_office_city',
 'registered_office_state',
 'role_in_firm',
 'role_on_application',
 'state',
 'utm_medium',
 'utm_source',
 'private_email_domain',
#  'age_of_firm',
#  'average_business_inflow',
#  'average_business_outflow',
#  'average_inflow',
#  'average_outflow',
#  'high_inflow_cp',
#  'inward_returns',
#  'max_business_inflow',
#  'max_business_outflow',
#  'max_inflow',
#  'max_outflow',
#  'min_business_inflow',
#  'min_business_outflow',
#  'min_inflow',
#  'min_outflow',
#  'outward_returns'
]

for col in categorical_cols:
    oh = pd.get_dummies(df[col], prefix=col)
    df = df.join(oh)
df = df.drop(columns=categorical_cols)

In [38]:
df

,amount,application_id,company_size,last_fy_profit,loan_created,age_of_firm,average_business_inflow,average_business_outflow,average_inflow,average_outflow,...,state_TELANGANA,state_UTTAR PRADESH,state_WEST BENGAL,state_madhya pradesh,utm_medium_Banner,utm_medium_ppc,utm_source_Facebook,utm_source_adwords,private_email_domain_False,private_email_domain_True
0,200000.0,1031,NaN,NaN,0,2.0,1.647250e+05,1.640400e+05,1.647250e+05,1.647250e+05,...,0,0,0,0,0,0,0,0,1,0
1,300000.0,2056,5.0,341068.0,0,4.0,9.122000e+04,9.162500e+04,9.122000e+04,9.122000e+04,...,0,0,0,0,0,1,0,1,1,0
2,500000.0,9047,NaN,NaN,0,4.0,4.212600e+04,4.729200e+04,4.212600e+04,4.212600e+04,...,0,0,0,0,1,0,1,0,1,0
3,500000.0,2068,5.0,NaN,0,4.0,2.466550e+05,2.644910e+05,2.466550e+05,2.466550e+05,...,0,0,0,0,0,1,0,1,1,0
4,500000.0,2737,5.0,NaN,0,4.0,9.678400e+04,9.549800e+04,9.678400e+04,9.678400e+04,...,0,0,0,0,0,1,0,1,1,0
5,400000.0,3126,5.0,NaN,0,5.0,6.500300e+04,6.717000e+04,6.500300e+04,6.500300e+04,...,0,0,0,0,0,1,0,1,1,0
6,600000.0,1081,5.0,319713.0,0,NaN,1.118550e+05,1.106320e+05,1.118550e+05,1.118550e+05,...,0,0,0,0,0,1,0,1,1,0
7,300000.0,5178,5.0,NaN,0,NaN,4.463800e+04,4.341500e+04,4.463800e+04,4.463800e+04,...,0,0,0,0,0,1,0,1,1,0
8,500000.0,1084,5.0,NaN,0,2.0,2.366100e+04,2.257600e+04,2.366100e+04,2.366100e+04,...,0,0,0,0,0,1,0,1,1,0
9,1000000.0,6281,NaN,NaN,0,3.0,2.279537e+05,2.259997e+05,2.279537e+05,2.279537e+05,...,0,0,1,0,1,0,1,0,0,1


In [39]:
df.to_csv(TRAIN_DATA, index=False)